# Section 1: Full Extraction

In [ ]:
!pip install pandas

In [3]:
import pandas as pd
from datetime import datetime

# Loading the entire CSV
df_full = pd.read_csv('custom_data.csv', parse_dates=['order_date'])

# Displaying basic information
print(f"Total Rows Extracted: {df_full.shape[0]}")
print(f"Total Columns: {df_full.shape[1]}")
print("\nSample of the data:")
display(df_full.head())

Total Rows Extracted: 100
Total Columns: 6

Sample of the data:


,order_id,customer_name,product,order_date,quantity,total_price
0,1,Customer 1,Tablet,2025-06-26 11:18:00,1,583.28
1,2,Customer 2,Camera,2025-06-13 11:27:00,4,2299.20
2,3,Customer 3,Headphones,2025-06-23 22:30:00,3,3682.14
3,4,Customer 4,Headphones,2025-06-10 14:30:00,1,687.12
4,5,Customer 5,Smartphone,2025-06-26 02:40:00,1,990.39


# Section 2: Partial Extraction

In [6]:
# Step 1: Reading the last extraction timestamp from file
with open('last_extraction.txt', 'r') as file:
    last_extraction_time_str = file.read().strip()  # Read and remove whitespace/newlines

# Converting the string to pandas Timestamp for comparison
last_extraction_time = pd.to_datetime(last_extraction_time_str)
print(f"Last Extraction Time: {last_extraction_time}")

# Step 2: Loading the dataset again, ensuring 'order_date' is parsed as datetime
df = pd.read_csv('custom_data.csv', parse_dates=['order_date'])

# Step 3: Filtering rows where 'order_date' is strictly greater than last extraction time
df_incremental = df[df['order_date'] > last_extraction_time]

# Printing how many new rows were extracted
print(f"New Records Extracted: {df_incremental.shape[0]}")

# Displaying the new rows for verification
display(df_incremental)

Last Extraction Time: 2025-06-12 10:05:31
New Records Extracted: 66


,order_id,customer_name,product,order_date,quantity,total_price
0,1,Customer 1,Tablet,2025-06-26 11:18:00,1,583.28
1,2,Customer 2,Camera,2025-06-13 11:27:00,4,2299.20
2,3,Customer 3,Headphones,2025-06-23 22:30:00,3,3682.14
4,5,Customer 5,Smartphone,2025-06-26 02:40:00,1,990.39
5,6,Customer 6,Camera,2025-07-05 00:43:00,4,3613.93
...,...,...,...,...,...,...
91,92,Customer 92,Camera,2025-06-12 10:11:00,2,1690.89
92,93,Customer 93,Camera,2025-06-15 16:15:00,2,1253.02
94,95,Customer 95,Headphones,2025-07-04 13:39:00,3,264.65
97,98,Customer 98,Camera,2025-06-17 18:44:00,3,4367.03


# Section 3: Saving new Timestamp

In [5]:
# Get current extraction time
current_time = datetime.now()

# Update the last_extraction.txt file
with open('last_extraction.txt', 'w') as file:
    file.write(current_time.strftime('%Y-%m-%d %H:%M:%S'))

print(f"Extraction timestamp updated to: {current_time}")

Extraction timestamp updated to: 2025-06-12 10:05:31.793731


# Section 4: Transform full data

In [1]:
import numpy as np

In [7]:
# 4.1: Cleaning: Remove duplicates (if any)

before_dups = df_full.shape[0]
df_full_clean = df_full.drop_duplicates()
after_dups = df_full_clean.shape[0]
print(f"Removed {before_dups - after_dups} duplicate rows from full data.")

Removed 0 duplicate rows from full data.


In [10]:
# 4.2 Cleaning: Handle missing values (if any)

missing_counts = df_full_clean.isna().sum()
print("Missing values per column in full data:" + f"\n{missing_counts}") 

Missing values per column in full data:
order_id         0
customer_name    0
product          0
order_date       0
quantity         0
total_price      0
dtype: int64


In [ ]:
# 4.3: Structural: Extract date parts

df_full_clean['order_date'] = pd.to_datetime(df_full_clean['order_date'], errors='coerce')

# Create year, month, day columns for analysis
df_full_clean['order_year'] = df_full_clean['order_date'].dt.year
df_full_clean['order_month'] = df_full_clean['order_date'].dt.month
df_full_clean['order_day'] = df_full_clean['order_date'].dt.day
print("Added order_year, order_month, order_day columns.")
display(df_full_clean.head())

Added order_year, order_month, order_day columns.


,order_id,customer_name,product,order_date,quantity,total_price,order_year,order_month,order_day
0,1,Customer 1,Tablet,2025-06-26 11:18:00,1,583.28,2025,6,26
1,2,Customer 2,Camera,2025-06-13 11:27:00,4,2299.20,2025,6,13
2,3,Customer 3,Headphones,2025-06-23 22:30:00,3,3682.14,2025,6,23
3,4,Customer 4,Headphones,2025-06-10 14:30:00,1,687.12,2025,6,10
4,5,Customer 5,Smartphone,2025-06-26 02:40:00,1,990.39,2025,6,26


In [ ]:
# 4.4: Enrichment: Calculate new metrics

df_full_clean['avg_price_per_item'] = df_full_clean['total_price'] / df_full_clean['quantity']
print("Added avg_price_per_item column.")

# Display the enriched data
display(df_full_clean.head())

Added avg_price_per_item column.


,order_id,customer_name,product,order_date,quantity,total_price,order_year,order_month,order_day,avg_price_per_item
0,1,Customer 1,Tablet,2025-06-26 11:18:00,1,583.28,2025,6,26,583.28
1,2,Customer 2,Camera,2025-06-13 11:27:00,4,2299.20,2025,6,13,574.80
2,3,Customer 3,Headphones,2025-06-23 22:30:00,3,3682.14,2025,6,23,1227.38
3,4,Customer 4,Headphones,2025-06-10 14:30:00,1,687.12,2025,6,10,687.12
4,5,Customer 5,Smartphone,2025-06-26 02:40:00,1,990.39,2025,6,26,990.39


In [16]:
# 4.5: Categorization: Bin avg_price_per_item into Low/Medium/High
# Define bins (adjust thresholds as appropriate for your data distribution)

bins = [ -np.inf, 500, 1000, np.inf ]
labels = ['Low', 'Medium', 'High']
df_full_clean['price_category'] = pd.cut(df_full_clean['avg_price_per_item'], bins=bins, labels=labels)
print("Added item_price_category column with categories Low/Medium/High.")

Added item_price_category column with categories Low/Medium/High.


In [17]:
# Display the categorized data
display(df_full_clean.head())

,order_id,customer_name,product,order_date,quantity,total_price,order_year,order_month,order_day,avg_price_per_item,price_category
0,1,Customer 1,Tablet,2025-06-26 11:18:00,1,583.28,2025,6,26,583.28,Medium
1,2,Customer 2,Camera,2025-06-13 11:27:00,4,2299.20,2025,6,13,574.80,Medium
2,3,Customer 3,Headphones,2025-06-23 22:30:00,3,3682.14,2025,6,23,1227.38,High
3,4,Customer 4,Headphones,2025-06-10 14:30:00,1,687.12,2025,6,10,687.12,Medium
4,5,Customer 5,Smartphone,2025-06-26 02:40:00,1,990.39,2025,6,26,990.39,Medium


In [ ]:
# 4.6: Filtering: Dropping irrelevant columns
# For analysis, maybe we don't need 'customer_name' since order_id can identify customers
if 'customer_name' in df_full_clean.columns:
    df_full_clean = df_full_clean.drop(columns=['customer_name'])
    print("Dropped 'customer_name' column.")

Dropped 'customer_name' column.


In [ ]:
# 4.7: Final transformed full data overview and Saving the transformed full data to CSV
print("Transformed full data shape:", df_full_clean.shape)
display(df_full_clean.head())

df_full_clean.to_csv('transformed_full.csv', index=False)
print("Saved transformed full data to 'transformed_full.csv'.")

Transformed full data shape: (100, 10)


,order_id,product,order_date,quantity,total_price,order_year,order_month,order_day,avg_price_per_item,price_category
0,1,Tablet,2025-06-26 11:18:00,1,583.28,2025,6,26,583.28,Medium
1,2,Camera,2025-06-13 11:27:00,4,2299.20,2025,6,13,574.80,Medium
2,3,Headphones,2025-06-23 22:30:00,3,3682.14,2025,6,23,1227.38,High
3,4,Headphones,2025-06-10 14:30:00,1,687.12,2025,6,10,687.12,Medium
4,5,Smartphone,2025-06-26 02:40:00,1,990.39,2025,6,26,990.39,Medium


Saved transformed full data to 'transformed_full.csv'.


# Section 5: Transform Incremental Data

In [20]:
# 5.1: Cleaning: Remove duplicates in incremental

before_inc_dups = df_incremental.shape[0]
df_inc_clean = df_incremental.drop_duplicates()
after_inc_dups = df_inc_clean.shape[0]
print(f"Removed {before_inc_dups - after_inc_dups} duplicate rows from incremental data.")

Removed 0 duplicate rows from incremental data.


In [ ]:
# 5.2: Cleaning: Handle missing values in incremental

missing_inc = df_inc_clean.isna().sum()
print("Missing values per column in incremental data:" + f"\n{missing_inc}")

Missing values per column in incremental data:
order_id         0
customer_name    0
product          0
order_date       0
quantity         0
total_price      0
dtype: int64


In [ ]:
# 5.3: Structural: Ensure datetime and extract parts

df_inc_clean['order_date'] = pd.to_datetime(df_inc_clean['order_date'], errors='coerce')

# Create year, month, day columns for analysis

df_inc_clean['order_year'] = df_inc_clean['order_date'].dt.year
df_inc_clean['order_month'] = df_inc_clean['order_date'].dt.month
df_inc_clean['order_day'] = df_inc_clean['order_date'].dt.day
print("Added order_year, order_month, order_day to incremental data.")

Added order_year, order_month, order_day to incremental data.


In [ ]:
# 5.4: Enrichment: Calculate avg_price_per_item
df_inc_clean['avg_price_per_item'] = df_inc_clean['total_price'] / df_inc_clean['quantity']
print("Added avg_price_per_item to incremental data.")

Added avg_price_per_item to incremental data.


In [24]:
# 5.5: Categorization: Bin avg_price_per_item into Low/Medium/High

df_inc_clean['price_category'] = pd.cut(df_inc_clean['avg_price_per_item'], bins=bins, labels=labels)
print("Added item_price_category column with categories Low/Medium/High.")

# Display the categorized data
display(df_inc_clean.head())

Added item_price_category column with categories Low/Medium/High.


,order_id,customer_name,product,order_date,quantity,total_price,order_year,order_month,order_day,avg_price_per_item,price_category
0,1,Customer 1,Tablet,2025-06-26 11:18:00,1,583.28,2025,6,26,583.2800,Medium
1,2,Customer 2,Camera,2025-06-13 11:27:00,4,2299.20,2025,6,13,574.8000,Medium
2,3,Customer 3,Headphones,2025-06-23 22:30:00,3,3682.14,2025,6,23,1227.3800,High
4,5,Customer 5,Smartphone,2025-06-26 02:40:00,1,990.39,2025,6,26,990.3900,Medium
5,6,Customer 6,Camera,2025-07-05 00:43:00,4,3613.93,2025,7,5,903.4825,Medium


In [25]:
# 5.6: Filtering: Dropping irrelevant columns in incremental
if 'customer_name' in df_inc_clean.columns:
    df_inc_clean = df_inc_clean.drop(columns=['customer_name'])
    print("Dropped 'customer_name' column from incremental data.")

Dropped 'customer_name' column from incremental data.


In [26]:
# 5.7: Final transformed incremental data overview and Saving the transformed incremental data to CSV
print("Transformed incremental data shape:", df_inc_clean.shape)
display(df_inc_clean.head())

df_inc_clean.to_csv('transformed_incremental.csv', index=False)
print("Saved transformed incremental data to 'transformed_incremental.csv'.")

Transformed incremental data shape: (66, 10)


,order_id,product,order_date,quantity,total_price,order_year,order_month,order_day,avg_price_per_item,price_category
0,1,Tablet,2025-06-26 11:18:00,1,583.28,2025,6,26,583.2800,Medium
1,2,Camera,2025-06-13 11:27:00,4,2299.20,2025,6,13,574.8000,Medium
2,3,Headphones,2025-06-23 22:30:00,3,3682.14,2025,6,23,1227.3800,High
4,5,Smartphone,2025-06-26 02:40:00,1,990.39,2025,6,26,990.3900,Medium
5,6,Camera,2025-07-05 00:43:00,4,3613.93,2025,7,5,903.4825,Medium


Saved transformed incremental data to 'transformed_incremental.csv'.
